<a href="https://colab.research.google.com/github/EjbejaranosAI/EmotionUnify/blob/main/Multibench_part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mosi dataset with head classifier

In [1]:
!git clone https://github.com/pliang279/MultiBench.git
%cd MultiBench
!pip install -q memory_profiler

Cloning into 'MultiBench'...
remote: Enumerating objects: 6931, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 6931 (delta 65), reused 121 (delta 60), pack-reused 6789
Receiving objects: 100% (6931/6931), 51.06 MiB | 11.15 MiB/s, done.
Resolving deltas: 100% (4251/4251), done.
/content/MultiBench


In [2]:
import os
import torch
import sys
import gdown
from torch import nn

# Create the directory if it doesn't exist
if not os.path.exists('data'):
    os.makedirs('data')

# Download the file to the directory
url = 'https://drive.google.com/u/0/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU'
output = 'data/mosi_raw.pkl'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU
To: /content/MultiBench/data/mosi_raw.pkl
100%|██████████| 357M/357M [00:04<00:00, 78.3MB/s]


'data/mosi_raw.pkl'

Importing the dataloaders for MOSI, and import that data using the path we stored the MOSI_RAW.pkl file to.

In [4]:
from datasets.affect.get_data import get_dataloader # noqa
traindata, validdata, testdata = \
    get_dataloader('/content/MultiBench/data/mosi_raw.pkl', robust_test=False)

Definning the encoder and decoder modules for each modality, taken from the associated section of MultiBench.

In [5]:
from unimodals.common_models import GRU, MLP # noqa
from fusions.MCTN import Encoder, Decoder # noqa


max_seq = 20
feature_dim = 300
hidden_dim = 32

encoder0 = Encoder(feature_dim, hidden_dim, n_layers=1, dropout=0.0).cuda()
decoder0 = Decoder(hidden_dim, feature_dim, n_layers=1, dropout=0.0).cuda()
encoder1 = Encoder(hidden_dim, hidden_dim, n_layers=1, dropout=0.0).cuda()
decoder1 = Decoder(hidden_dim, feature_dim, n_layers=1, dropout=0.0).cuda()

reg_encoder = nn.GRU(hidden_dim, 32).cuda()

Defining the classification head for our model:

In [6]:
from unimodals.common_models import MLP # noqa
head = MLP(32, 64, 1).cuda()

In [7]:
from private_test_scripts.all_in_one import all_in_one_train # noqa
from training_structures.MCTN_Level2 import train, test # noqa

allmodules = [encoder0, decoder0, encoder1, decoder1, reg_encoder, head]


def trainprocess():
    train(
        traindata, validdata,
        encoder0, decoder0, encoder1, decoder1,
        reg_encoder, head,
        criterion_t0=nn.MSELoss(), criterion_c=nn.MSELoss(),
        criterion_t1=nn.MSELoss(), criterion_r=nn.L1Loss(),
        max_seq_len=20,
        mu_t0=0.01, mu_c=0.01, mu_t1=0.01,
        dropout_p=0.15, early_stop=False, patience_num=15,
        lr=1e-4, weight_decay=0.01, op_type=torch.optim.AdamW,
        epoch=200, model_save='best_mctn.pt')


all_in_one_train(trainprocess, allmodules)

model = torch.load('best_mctn.pt').cuda()

test(model, testdata, 'mosi', no_robust=True)

start training ---------->>
Train Epoch 0, total loss: 3.07175874710083, regression loss: 1.3455722332000732, embedding loss: 1.7261865139007568
Start Evaluating ---------->>
Eval Epoch: 0, MAE: 1.416121482849121, Acc1: 0.5747663551401869, Acc2: 0.6119402985074627
<------------ Saving Best Model

start training ---------->>
Train Epoch 1, total loss: 3.0539612770080566, regression loss: 1.335289716720581, embedding loss: 1.7186715602874756
Start Evaluating ---------->>
Eval Epoch: 1, MAE: 1.414206624031067, Acc1: 0.5747663551401869, Acc2: 0.6119402985074627
<------------ Saving Best Model

start training ---------->>
Train Epoch 2, total loss: 3.014385461807251, regression loss: 1.2960611581802368, embedding loss: 1.7183243036270142
Start Evaluating ---------->>
Eval Epoch: 2, MAE: 1.412585735321045, Acc1: 0.5747663551401869, Acc2: 0.6119402985074627
<------------ Saving Best Model

start training ---------->>
Train Epoch 3, total loss: 3.0526857376098633, regression loss: 1.3309831619